In [19]:
# ============================================================
# NHIP: Combined violin plots across tissue/region categories
#   - Two plots: promoter vs gene body
#   - Two violins per x-category: Control vs Other
#   - brain + cfDNA restricted to Obese vs Control only
# ============================================================

import os, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import matplotlib

# --- Make PDFs editable in Illustrator / keep vectors clean ---
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rcParams['figure.dpi'] = 300
matplotlib.rcParams['savefig.transparent'] = True
matplotlib.rcParams['path.simplify'] = False  # keep exact vectors


1- Define the genomic region + promoter/gene body windows

In [20]:
# -------------------- Common region definitions --------------------
chrom = "chr10"
region_start = 2432261
region_end   = 2443263

# "Promoter" defined as CpG island interval (as in your scripts)
cpg_start = 2433175
cpg_end   = 2433562

# Gene body window (consistent with cumulus/oocyte scripts you shared)
gene_body_start = region_start + 1000
gene_body_end   = region_end - 1000


2- Helper A — read ROI/cov-like files and compute methylation fraction per position

In [21]:
# -------------------- Helpers --------------------
def roi_frac_series_from_covlike_tsv_gz(path: str, chrom: str) -> pd.Series:
    """
    Reads ROI/cov-like tsv.gz with columns:
      chrom start end pct meth unmeth
    Returns:
      pd.Series of methylation fraction (meth/(meth+unmeth)),
      indexed by start position.
    """
    cols = ["chrom", "start", "end", "pct", "meth", "unmeth"]
    df = pd.read_csv(path, sep="\t", header=None, names=cols, compression="gzip")

    if df.empty:
        return pd.Series(dtype=float)

    df = df[df["chrom"] == chrom].copy()
    if df.empty:
        return pd.Series(dtype=float)

    denom = (df["meth"] + df["unmeth"]).replace(0, np.nan)
    df["frac"] = df["meth"] / denom
    df = df.dropna(subset=["frac"])

    return pd.Series(df["frac"].values, index=df["start"].astype(int).values)


3- Helper B — summarize “promoter mean” + “gene body mean” from a Series


In [22]:
def summarize_series_by_windows(s: pd.Series):
    """
    Given series:
      index = pos, values = methylation fraction
    Return:
      (promoter_mean, genebody_mean) for the predefined windows.
    """
    prom = s.loc[(s.index >= cpg_start) & (s.index <= cpg_end)]
    body = s.loc[(s.index >= gene_body_start) & (s.index <= gene_body_end)]

    promoter_mean = float(np.nanmean(prom.values)) if len(prom) else np.nan
    genebody_mean = float(np.nanmean(body.values)) if len(body) else np.nan
    return promoter_mean, genebody_mean


4- Helper C — summarize “promoter mean” + “gene body mean” from cumulus/oocyte TSVs

In [23]:
def summarize_pos_meth_df(df: pd.DataFrame):
    """
    df must have columns:
      pos (int), meth (float)
    """
    prom = df[(df["pos"] >= cpg_start) & (df["pos"] <= cpg_end)]["meth"]
    body = df[(df["pos"] >= gene_body_start) & (df["pos"] <= gene_body_end)]["meth"]

    promoter_mean = float(np.nanmean(prom.values)) if len(prom) else np.nan
    genebody_mean = float(np.nanmean(body.values)) if len(body) else np.nan
    return promoter_mean, genebody_mean


5- Helper D — collapse many group labels into 2 groups: Control vs Other

In [24]:
def normalize_groups_to_control_other(df: pd.DataFrame, control_names=("Control","C")) -> pd.DataFrame:
    """
    Adds column 'grp2' with values:
      - 'Control' if df['group'] in control_names
      - 'Other' otherwise
    """
    df = df.copy()
    df["grp2"] = np.where(df["group"].isin(control_names), "Control", "Other")
    return df


6- Collectors 
6.1) Collector — Brain (Obese vs Control only)


In [25]:
def collect_brain_ovc_region(region_name: str) -> pd.DataFrame:
    PROJECT = Path("/quobyte/lasallegrp/Ensi/project/nhip_macaque/fromBen_MacaqueObese_brain/")
    DMRS_DIR = PROJECT / "DMRs"
    METADATA = DMRS_DIR / "sample_info_master.csv"  # tab-separated

    meta = pd.read_csv(METADATA, sep="\t")
    meta["Name"] = meta["Name"].astype(str)

    # Use Obesity status column, not treatment Group
    cond = meta["Obese"].astype(str)
    meta["Condition"] = np.where(cond.str.lower() == "obese", "Obese", "Control")
    name_to_condition = dict(zip(meta["Name"], meta["Condition"]))

    dmr_dir = DMRS_DIR / f"{region_name}_OvC"
    if not dmr_dir.exists():
        print(f"[brain] Missing folder: {dmr_dir}")
        return pd.DataFrame(columns=["x_label","sample","group","promoter_mean","genebody_mean"])

    roi_files = sorted(glob.glob(str(dmr_dir / "*.roi.tsv.gz")))
    cov_files = sorted(glob.glob(str(dmr_dir / "*.cov.gz")))
    files_to_use = roi_files if roi_files else cov_files

    rows = []
    for fpath in files_to_use:
        base = os.path.basename(fpath)
        sample = base.split("_", 1)[0]  # e.g., 46505-Hippocampus

        if sample not in name_to_condition:
            continue

        group = name_to_condition[sample]  # Control or Obese
        if group not in {"Control", "Obese"}:
            continue

        s = roi_frac_series_from_covlike_tsv_gz(fpath, chrom=chrom)
        if s.empty:
            continue

        prom_mean, body_mean = summarize_series_by_windows(s)

        rows.append({
            "x_label": f"brain:{region_name}",
            "sample": sample,
            "group": group,
            "promoter_mean": prom_mean,
            "genebody_mean": body_mean
        })

    return pd.DataFrame(rows)


6.2) Collector — cfDNA (Obese vs Control only)

In [26]:
def collect_cfdna_gd_ovc(gd_label: str) -> pd.DataFrame:
    """
    cfDNA: folders like GD45_OvC
    ROI files: *.roi.tsv.gz
    Sample = filename without .roi.tsv.gz
    """
    PROJECT = Path("/quobyte/lasallegrp/Ensi/project/nhip_macaque/fromBen_MacaqueObese_cfDna/")
    DMRS_DIR = PROJECT / "DMRs"
    METADATA = DMRS_DIR / "master_sample_info_cfDNA.csv"

    meta = pd.read_csv(METADATA)
    meta["Name"] = meta["Name"].astype(str)
    meta["Group"] = meta["Group"].astype(str)
    name_to_group = dict(zip(meta["Name"], meta["Group"]))

    dmr_dir = DMRS_DIR / f"{gd_label}_OvC"
    if not dmr_dir.exists():
        print(f"[cfDNA] Missing folder: {dmr_dir}")
        return pd.DataFrame(columns=["x_label","sample","group","promoter_mean","genebody_mean"])

    roi_files = sorted(glob.glob(str(dmr_dir / "*.roi.tsv.gz")))

    rows = []
    for fpath in roi_files:
        sample = os.path.basename(fpath).replace(".roi.tsv.gz", "")
        if sample not in name_to_group:
            continue

        group = name_to_group[sample]
        if group not in {"Control", "Obese"}:
            continue

        s = roi_frac_series_from_covlike_tsv_gz(fpath, chrom=chrom)
        if s.empty:
            continue

        prom_mean, body_mean = summarize_series_by_windows(s)

        rows.append({
            "x_label": f"cfDNA:{gd_label}",
            "sample": sample,
            "group": group,
            "promoter_mean": prom_mean,
            "genebody_mean": body_mean
        })

    return pd.DataFrame(rows)


6.3) Collector — nasal ROI (Control vs Exposed)

In [27]:
def collect_nasal_roi() -> pd.DataFrame:
    PROJECT = Path("/quobyte/lasallegrp/Ensi/project/nhip_macaque/macaque_nasal_HongJi/")
    OUT_DIR  = PROJECT / "roi_tsvfiles"
    METADATA = PROJECT / "sample_info_master.csv"  # comma-separated

    meta = pd.read_csv(METADATA, sep=",")
    meta["Name"] = meta["Name"].astype(str)
    meta["Group"] = meta["Group"].astype(str)
    name_to_group = dict(zip(meta["Name"], meta["Group"]))

    rows = []
    for fpath in sorted(glob.glob(str(OUT_DIR / "*.tsv.gz"))):
        base = os.path.basename(fpath)
        sample = base.split("_", 1)[0]

        if sample not in name_to_group:
            continue

        group = name_to_group[sample]
        if group not in {"Control", "Exposed"}:
            continue

        s = roi_frac_series_from_covlike_tsv_gz(fpath, chrom=chrom)
        if s.empty:
            continue

        prom_mean, body_mean = summarize_series_by_windows(s)

        rows.append({
            "x_label": "nasal:ROI",
            "sample": sample,
            "group": group,
            "promoter_mean": prom_mean,
            "genebody_mean": body_mean
        })

    return pd.DataFrame(rows)


6.4) Collector — cumulus ROI (pos/type/meth TSV)


In [14]:
def collect_cumulus_roi() -> pd.DataFrame:
    ROOT = Path("/quobyte/lasallegrp/Ensi/project/nhip_macaque/nhip_cumulus/")
    META = ROOT / "sample_info.csv"   # tab-separated
    DATA_DIR = ROOT / "roi_tsvfiles"

    meta = pd.read_csv(META, sep="\t")
    meta["Name"] = meta["Name"].astype(str)
    meta["Group"] = meta["Group"].astype(str)
    sample_to_group = dict(zip(meta["Name"], meta["Group"]))

    rows = []
    for fpath in sorted(glob.glob(str(DATA_DIR / "*_merged_name_sorted.deduplicated.bismark.cov.gz.CpG_report.merged_CpG_evidence.cov.gz.chr.txt.gz.roi1kb.tsv.gz"))):
        sample = os.path.basename(fpath).replace("_merged_name_sorted.deduplicated.bismark.cov.gz.CpG_report.merged_CpG_evidence.cov.gz.chr.txt.gz.roi1kb.tsv.gz", "")

        if sample not in sample_to_group:
            continue

        group = sample_to_group[sample]
        
        
        s = roi_frac_series_from_covlike_tsv_gz(fpath, chrom=chrom)
        if s.empty:
            continue

        prom_mean, body_mean = summarize_series_by_windows(s)

        rows.append({
            "x_label": "oocytes:ROI",
            "sample": sample,
            "group": group,
            "promoter_mean": prom_mean,
            "genebody_mean": body_mean
        })

    return pd.DataFrame(rows)

6.5) Collector — oocytes ROI (group inferred from filename suffix)

In [15]:
def collect_oocytes_roi() -> pd.DataFrame:
    ROOT = Path("/quobyte/lasallegrp/Ensi/project/nhip_macaque/nhip_oocytes/")
    DATA_DIR = ROOT / "roi_tsvfiles"

    rows = []
    for fpath in sorted(glob.glob(str(DATA_DIR / "*.CpGreport.txt.gz.CpGreport.mergedCpGevidence.roi1kb.tsv.gz"))):
        fname = os.path.basename(fpath)
        sample = fname.replace(".CpGreport.txt.gz.CpGreport.mergedCpGevidence.roi1kb.tsv.gz", "")

        # infer group from filename
        if sample.endswith("C"):
            group = "Control"
        elif sample.endswith("S"):
            group = "Stress"
        else:
            continue

        s = roi_frac_series_from_covlike_tsv_gz(fpath, chrom=chrom)
        if s.empty:
            continue

        prom_mean, body_mean = summarize_series_by_windows(s)

        rows.append({
            "x_label": "oocytes:ROI",
            "sample": sample,
            "group": group,
            "promoter_mean": prom_mean,
            "genebody_mean": body_mean
        })

    return pd.DataFrame(rows)


7- Plotting — build “two-group violins” + dots
7.1) Small helper for jitter (so dots don’t overlap)

In [16]:
def jitter(values, scale=0.04):
    return values + np.random.uniform(-scale, scale, size=len(values))


7.2) Main plotting function

In [17]:
def plot_two_group_violins(df: pd.DataFrame, value_col: str, out_png: str, title: str):
    # 1) drop missing values for the chosen y-variable
    df = df.dropna(subset=[value_col]).copy()
    if df.empty:
        print(f"No data to plot for {value_col}")
        return

    # 2) collapse groups into Control vs Other
    df = normalize_groups_to_control_other(df, control_names=("Control","C"))

    # 3) define x-axis order (IMPORTANT: controls the layout)
    x_levels = [
        "brain:Hypothalamus",
        "brain:Hippocampus",
        "brain:PrefrontalCortex",
        "cfDNA:GD45",
        "cfDNA:GD90",
        "cfDNA:GD120",
        "cfDNA:GD150",
        "nasal:ROI",
        "cumulus:ROI",
        "oocytes:ROI",
    ]

    # 4) pretty labels for plotting
    x_labels_pretty = [
        "Brain\nHypothalamus",
        "Brain\nHippocampus",
        "Brain\nPrefrontal\nCortex",
        "cfDNA\nGD45",
        "cfDNA\nGD90",
        "cfDNA\nGD120",
        "cfDNA\nGD150",
        "Nasal\nROI",
        "Cumulus\nROI",
        "Oocytes\nROI",
    ]

    # 5) keep only those categories
    df = df[df["x_label"].isin(x_levels)]

    # 6) create figure
    fig, ax = plt.subplots(figsize=(20, 7))

    base_positions = np.arange(len(x_levels))
    width = 0.38
    offsets = {"Control": -width/2, "Other": width/2}

    colors = {
        "Control": "#4C72B0",
        "Other":   "#DD8452",
    }

    # 7) draw one set of violins for each group
    for grp in ["Control", "Other"]:
        plot_data = []
        plot_pos = []

        # gather data category-by-category
        for i, x in enumerate(x_levels):
            vals = df[(df["x_label"] == x) & (df["grp2"] == grp)][value_col].values
            vals = vals[~np.isnan(vals)]
            if len(vals) == 0:
                continue

            plot_data.append(vals)
            plot_pos.append(base_positions[i] + offsets[grp])

        # draw violins + medians + dots
        if len(plot_data) > 0:
            vp = ax.violinplot(
                plot_data,
                positions=plot_pos,
                widths=width,
                showmeans=False,
                showmedians=True,
                showextrema=False
            )

            for body in vp["bodies"]:
                body.set_facecolor(colors[grp])
                body.set_edgecolor("black")
                body.set_alpha(0.6)
                body.set_linewidth(0.8)

            vp["cmedians"].set_color("black")
            vp["cmedians"].set_linewidth(2)

            # dots per sample
            for x_pos, vals in zip(plot_pos, plot_data):
                ax.scatter(
                    jitter(np.full(len(vals), x_pos)),
                    vals,
                    s=24,
                    color=colors[grp],
                    edgecolor="black",
                    linewidth=0.5,
                    alpha=0.9,
                    zorder=3
                )

    # 8) style axes
    ax.set_xticks(base_positions)
    ax.set_xticklabels(x_labels_pretty, fontsize=14, fontweight="bold")
    ax.set_ylabel("Mean methylation level", fontsize=15, weight="bold")
    ax.set_title(title, fontsize=18, weight="bold", pad=17)
    ax.set_ylim(0, 1.02)
    ax.tick_params(axis="y", labelsize=13)

    # 9) legend
    from matplotlib.patches import Patch
    legend_elements = [
        Patch(facecolor=colors["Control"], edgecolor="black", label="Control"),
        Patch(facecolor=colors["Other"], edgecolor="black", label="Condition"),
    ]
    ax.legend(
        handles=legend_elements,
        loc="upper left",
        bbox_to_anchor=(1.01, 1),
        frameon=False,
        fontsize=14
    )

    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

    # 10) save + show
    plt.tight_layout()
    plt.savefig(out_png, dpi=300)
    plt.show()
    print("Saved:", out_png)


8- Main workflow — collect everything → save table → plot promoter + gene body

In [18]:
def main():
    dfs = []

    # 1) Brain: exactly these 3 regions, OvC only
    for region in ["Hypothalamus", "Hippocampus", "PrefrontalCortex"]:
        dfs.append(collect_brain_ovc_region(region))

    # 2) cfDNA: exactly these GD folders, OvC only
    for gd in ["GD45", "GD90", "GD120", "GD150"]:
        dfs.append(collect_cfdna_gd_ovc(gd))

    # 3) Single-ROI tissues
    dfs.append(collect_nasal_roi())
    dfs.append(collect_cumulus_roi())
    dfs.append(collect_oocytes_roi())

    # 4) Combine into one table
    df_all = pd.concat(dfs, ignore_index=True)

    # 5) Save combined summary
    out_csv = "NHIP_10categories_promoter_genebody_means.csv"
    df_all.to_csv(out_csv, index=False)
    print("Saved:", out_csv)

    # 6) Plot promoter means
    plot_two_group_violins(
        df_all, "promoter_mean",
        out_png="NHIP_10categories_violin_promoter.pdf",
        title="NHIP Region - CpG island mean methylation — Control vs Condition"
    )

    # 7) Plot gene body means
    plot_two_group_violins(
        df_all, "genebody_mean",
        out_png="NHIP_10categories_violin_genebody.pdf",
        title="NHIP Region - Gene body mean methylation — Control vs Condition"
    )

if __name__ == "__main__":
    main()


KeyboardInterrupt: 

In [ ]:
df = collect_oocytes_roi()
df

In [37]:
df_all = pd.read_csv("NHIP_10categories_promoter_genebody_means.csv")
df_all

,x_label,sample,group,promoter_mean,genebody_mean
0,brain:Hypothalamus,46505-Hypothalamus,Obese,0.005952,0.752894
1,brain:Hypothalamus,46658-Hypothalamus,Control,0.102381,0.766342
2,brain:Hypothalamus,46668-Hypothalamus,Control,0.000000,0.746836
3,brain:Hypothalamus,46671-Hypothalamus,Obese,0.002646,0.760396
4,brain:Hypothalamus,46742-Hypothalamus,Control,0.023339,0.743621
...,...,...,...,...,...
278,oocytes:ROI,4660644938SO2NC1N9C,Control,NaN,0.800000
279,oocytes:ROI,4660644938SO3NC1O9C,Control,NaN,0.736842
280,oocytes:ROI,4660644993PO2NC5D10C,Control,NaN,0.696429
281,oocytes:ROI,4660644993PO3NC5E10C,Control,NaN,0.820513
